## Load the Motion Data

Data is in x and y coordinates for each pixel. Each sample will be an array of 10 (frames) x 40 x 40 (capture window) x 2 (x and y) dimensions. 

In [1]:
import numpy as np
import os
import matplotlib
matplotlib.use('TkAgg') # For displaying animation
import matplotlib.pyplot as plt
import matplotlib.animation as animation

from helper_functions import *
from normalize_data import *
from integral_try import *
from numpy import array
from sklearn import preprocessing as pp

normalization function imported
Integral functions imported


In [2]:
data_dir = 'sonic_pi_face/data/'

# Get list of data files
data_files = get_data_files(data_dir)

# Load data into a dictionary
# Note: Checks for incomplete data
data = get_gesture_data(data_files)

# Get number of `open-close` samples
gesture_type = 'open-close'
print("Samples \'{}\': {}".format(gesture_type, len(data[gesture_type])))

Incomplete data sample found at  /Users/justinshenk/Projects/dance/sonic_pi_face/data/2017-05-30_130721_open-close.txt
Incomplete data sample found at  /Users/justinshenk/Projects/dance/sonic_pi_face/data/2017-05-30_131354_open-close.txt
Incomplete data sample found at  /Users/justinshenk/Projects/dance/sonic_pi_face/data/2017-05-30_163658_open-close.txt
Incomplete data sample found at  /Users/justinshenk/Projects/dance/sonic_pi_face/data/2017-05-30_164242_open-close.txt
Incomplete data sample found at  /Users/justinshenk/Projects/dance/sonic_pi_face/data/2017-05-30_164306_open-close.txt
Incomplete data sample found at  /Users/justinshenk/Projects/dance/sonic_pi_face/data/2017-05-30_164406_open-close.txt
Samples 'open-close': 134


## Normalize data sample and get integral image for feature detection

In [ ]:
# Normalize data across data samples and frames (x and y will be normalized independently)
# data = normalize_data(data)
print(np.shape(data['open-close']))

# Load first data sample from `open-close`
sample = data['open-close'][0]
print(np.shape(sample))
print(np.max(sample))

# calculate the integral image
iimage = i_image(sample[4])

# calculate the integral for the first quarter from the integral image
try_integral = get_integral(iimage,0,0,20,20)

print('Example integral of the 1st quarter of one frame: ' , try_integral)

## Visualize horizontal motion across frames

In [4]:
sample = data['open-close'][36] # Fifth sample
frame = sample[6][:,:,0] # Middle frame
# Note: Second argument takes 0 for `x`, 1 for `y` coordinate
# display_frames(sample,0) 

## Random forest classification

In [45]:
open_close = data['open-close']
oc_array = np.asarray(open_close)
sample_no = 0
frame_no = 0
oc_array[sample_no][frame_no][...,0]
# flatten array to n x 32000
flattened = oc_array.reshape((len(oc_array),10*1600*2))

In [83]:
from sklearn.ensemble import RandomForestClassifier
dataX = flattened.copy()

# create random mock data on same scale as original data
shamX = np.random.randn(*dataX.shape) * (np.max(dataX) + np.abs(np.min(dataX))) - np.min(dataX)
X = np.vstack((dataX, shamX))
dataY = np.ones((len(dataX)))
shamY = np.zeros((len(shamX)))
Y = np.hstack((dataY, shamY))

# split data into train and test
from sklearn.model_selection import train_test_split

In [84]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state=42)
clf = RandomForestClassifier(n_estimators=10)
clf = clf.fit(X_train, y_train)
clf.score(X_train,y_train)

1.0